In [68]:
import tkinter as tk
from tkinter import ttk, messagebox
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import networkx as nx
import numpy as np
import time

from ga import run_genetic_algorithm
from graph_utils import create_random_graph
from metrics import compute_metrics


# ===============================================================
# BACKEND
# ===============================================================
class IntegratedBackend:
    def __init__(self):
        self.G = create_random_graph(
            250, 0.03,
            edge_file="C:\\Users\\ASUS\\Desktop\\bilgisayar_aglari\\BSM307_317_Guz2025_TermProject_EdgeData (1).csv",
            demand_file="C:\\Users\\ASUS\\Desktop\\bilgisayar_aglari\\BSM307_317_Guz2025_TermProject_DemandData (1).csv",
            node_file="C:\\Users\\ASUS\\Desktop\\bilgisayar_aglari\\BSM307_317_Guz2025_TermProject_NodeData (1).csv"
        )

        pos = nx.spring_layout(self.G, seed=42)
        nx.set_node_attributes(self.G, pos, "pos")

    def get_path_results(self, s, d, talep, weights):
        start = time.perf_counter()

        best_path, best_cost = run_genetic_algorithm(
            self.G, s, d, talep, weights
        )

        elapsed = time.perf_counter() - start

        if not best_path:
            return None

        raw = compute_metrics(self.G, best_path)

        metrics = {
            "total_delay": raw["total_delay"],
            "total_reliability": raw["total_reliability"],
            "total_resource": raw["resource_cost"],
            "total_cost": best_cost,
            "time": elapsed
        }

        return {"path": best_path, "metrics": metrics}


# ===============================================================
# UI
# ===============================================================
class RoutingInterface:
    def __init__(self, root):
        self.root = root
        self.root.title("QoS Tabanlı Çok Amaçlı Rotalama Analiz Paneli")
        self.root.geometry("1400x900")
        self.root.configure(bg="#f8f9fa")

        self.backend = IntegratedBackend()
        self.setup_ui()

    # ---------------- UI ----------------
    def setup_ui(self):
        self.left_panel = tk.Frame(self.root, width=260, bg="white", padx=10)
        self.left_panel.pack_propagate(False)
        self.left_panel.pack(side=tk.LEFT, fill=tk.Y)

        tk.Label(self.left_panel, text="Kullanıcı Girişleri",
                 font=("Segoe UI", 16, "bold"), bg="white").pack(pady=15)

        self.create_label("Kaynak Düğüm (S)")
        self.src_var = tk.IntVar(value=0)
        tk.Spinbox(self.left_panel, from_=0, to=249,
                   textvariable=self.src_var, width=10).pack(anchor="w")

        self.create_label("Hedef Düğüm (D)")
        self.dst_var = tk.IntVar(value=10)
        tk.Spinbox(self.left_panel, from_=0, to=249,
                   textvariable=self.dst_var, width=10).pack(anchor="w")

        self.create_label("Talep (Bandwidth)")
        self.demand_var = tk.DoubleVar(value=5)
        tk.Entry(self.left_panel, textvariable=self.demand_var).pack(fill=tk.X)

        tk.Label(self.left_panel, text="\nOptimizasyon Ağırlıkları",
                 font=("Segoe UI", 11, "bold"), bg="white").pack(anchor="w")

        self.w1 = self.create_weight_slider("Gecikme", 0.4)
        self.w2 = self.create_weight_slider("Güvenilirlik", 0.3)
        self.w3 = self.create_weight_slider("Kaynak", 0.3)

        tk.Button(self.left_panel, text="EN İYİ ROTAYI HESAPLA",
                  bg="#20c997", fg="white",
                  font=("Segoe UI", 11, "bold"),
                  command=self.calculate).pack(fill=tk.X, pady=15)

        tk.Label(self.left_panel, text="Sonuç Detayı",
                 font=("Segoe UI", 11, "bold"), bg="white").pack(anchor="w")

        self.result_text = tk.Text(self.left_panel, height=12, wrap=tk.WORD)
        self.result_text.pack(fill=tk.BOTH)

        # ---------------- RIGHT ----------------
        self.right_panel = tk.Frame(self.root, bg="#f8f9fa", padx=20, pady=20)
        self.right_panel.pack(side=tk.RIGHT, expand=True, fill=tk.BOTH)

        self.metrics_frame = tk.Frame(self.right_panel, bg="#f8f9fa")
        self.metrics_frame.pack(fill=tk.X)

        self.cards = {
            "Delay": self.create_card("Ort. Gecikme", "0 ms", "#ff6b6b"),
            "Rel": self.create_card("Güvenilirlik", "%0", "#51cf66"),
            "Res": self.create_card("Kaynak", "0", "#339af0")
        }

        self.fig = plt.Figure(figsize=(10, 5))
        self.ax_net = self.fig.add_subplot(121)
        self.ax_radar = self.fig.add_subplot(122, polar=True)

        self.canvas = FigureCanvasTkAgg(self.fig, master=self.right_panel)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)

    # ---------------- YARDIMCI ----------------
    def create_label(self, text):
        tk.Label(self.left_panel, text=text,
                 bg="white", font=("Segoe UI", 10)).pack(anchor="w", pady=(8, 0))

    def create_weight_slider(self, label, default):
        tk.Label(self.left_panel, text=label, bg="white").pack(anchor="w")
        var = tk.DoubleVar(value=default)
        tk.Scale(self.left_panel, from_=0, to=1, resolution=0.1,
                 variable=var, orient=tk.HORIZONTAL,
                 bg="white", length=220).pack()
        return var

    def create_card(self, title, value, color):
        card = tk.Frame(self.metrics_frame, bg="white", padx=20, pady=15)
        card.pack(side=tk.LEFT, expand=True, fill=tk.X, padx=5)
        tk.Label(card, text=title, bg="white", fg="#868e96").pack()
        lbl = tk.Label(card, text=value, bg="white",
                       fg=color, font=("Segoe UI", 14, "bold"))
        lbl.pack()
        return lbl

    def get_weights(self):
        w1, w2, w3 = self.w1.get(), self.w2.get(), self.w3.get()
        total = w1 + w2 + w3
        if total == 0:
            messagebox.showerror("Hata", "Ağırlıklar toplamı 0 olamaz")
            return None
        return {"delay": w1/total, "reliability": w2/total, "resource": w3/total}

    # ---------------- RADAR ----------------
    def draw_radar(self, m):
        self.ax_radar.clear()
        labels = ["Delay", "Reliability", "Resource"]
        values = [m["total_delay"]/100, m["total_reliability"], m["total_resource"]/100]
        angles = np.linspace(0, 2*np.pi, len(labels), endpoint=False)
        values += values[:1]
        angles = np.append(angles, angles[0])
        self.ax_radar.plot(angles, values, linewidth=2)
        self.ax_radar.fill(angles, values, alpha=0.3)
        self.ax_radar.set_thetagrids(np.degrees(angles[:-1]), labels)

    # ---------------- HESAPLAMA ----------------
    def calculate(self):
        s, d = self.src_var.get(), self.dst_var.get()

        if s not in self.backend.G or d not in self.backend.G:
            messagebox.showwarning("Uyarı", "Seçilen S veya D graf içinde yok!")
            return

        if s == d:
            messagebox.showwarning("Uyarı", "S ve D aynı olamaz")
            return

        weights = self.get_weights()
        if weights is None:
            return

        results = self.backend.get_path_results(s, d, self.demand_var.get(), weights)
        if results is None:
            messagebox.showinfo("Sonuç", "Uygun yol bulunamadı")
            return

        m, path = results["metrics"], results["path"]

        # ---- KARTLAR ----
        self.cards["Delay"].config(text=f"{m['total_delay']:.2f} ms")
        self.cards["Rel"].config(text=f"%{m['total_reliability']*100:.1f}")
        self.cards["Res"].config(text=f"{m['total_resource']:.1f}")

        # ---- METİN ----
        self.result_text.delete("1.0", tk.END)
        self.result_text.insert(tk.END, f"Yol:\n{path}\n\n")
        for k, v in m.items():
            self.result_text.insert(tk.END, f"{k}: {v}\n")

        # ---- GRAF ----
        self.ax_net.clear()
        pos = nx.get_node_attributes(self.backend.G, "pos")

        nx.draw(self.backend.G, pos, ax=self.ax_net,
                node_size=8, node_color="#ced4da", edge_color="#dee2e6")

        edges = list(zip(path, path[1:]))
        nx.draw_networkx_edges(self.backend.G, pos, edgelist=edges,
                               edge_color="#ff6b6b", width=2.5, ax=self.ax_net)

        nx.draw_networkx_nodes(self.backend.G, pos, nodelist=path,
                               node_color="#ff6b6b", node_size=70, ax=self.ax_net)

        nx.draw_networkx_nodes(self.backend.G, pos, nodelist=[s],
                               node_color="#51cf66", node_size=120, ax=self.ax_net)
        nx.draw_networkx_nodes(self.backend.G, pos, nodelist=[d],
                               node_color="#845ef7", node_size=120, ax=self.ax_net)

        self.ax_net.set_title(f"Genetik Algoritma ({m['time']:.3f} sn)")
        self.draw_radar(m)
        self.canvas.draw()


# ===============================================================
if __name__ == "__main__":
    root = tk.Tk()
    RoutingInterface(root)
    root.mainloop()